In [12]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

In [13]:
# https://www.genie.co.kr/chart/top200?ditc=D&ymd=20230424&hh=13&rtm=Y&pg=1
url = 'https://www.genie.co.kr/chart/top200'
result = requests.get(url)
result.text

'<br>\r\n<br>\r\n<center>\r\n<img src="http://www.geniemusic.co.kr/images/common/logo_r1.png"><br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [14]:
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
result = requests.get(url, headers=header)
soup = BeautifulSoup(result.text,'html.parser')

In [15]:
trs = soup.select('tr.list')
len(trs)

50

In [16]:
# #body-content > div.newest-list > div > table > tbody > tr:nth-child(1)
trs = soup.select('#body-content > div.newest-list > div > table > tbody > tr')
len(trs)

50

In [17]:
# rank,img_url,title,artist,album
tr = trs[0]
tr

<tr class="list" songid="101268096">
<td class="check"><input class="select-check" title="I AM" type="checkbox"/></td>
<td class="number">1
                                        
                                    <span class="rank">
<span class="rank"><span class="rank-none"><span class="hide">유지</span></span></span>
</span>
</td>
<td><a class="cover" href="#" onclick="fnViewAlbumLayer('83665559');return false;"><span class="mask"></span><img alt="I've IVE" onerror="this.src='//image.genie.co.kr/imageg/web/common/blank_68.gif';" src="//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/083/665/559/83665559_1681116738103_1_140x140.JPG/dims/resize/Q_80,0"/></a></td>
<td class="link"><a class="btn-basic btn-info" href="#" onclick="fnViewSongInfo('101268096');return false;">곡 제목 정보 페이지</a></td>
<td class="info">
<a class="title ellipsis" href="#" onclick="fnPlaySong('101268096','1');return false;" title="재생">
                                        
                                            
      

In [24]:
# rank
tr.select_one('.number').get_text().split('\n')[0]

'1'

In [26]:
rank = tr.select_one('.number').get_text().split('\n')[0].strip()
rank

'1'

In [32]:
# img_url
tr.select_one('.cover > img')['src']

'//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/083/665/559/83665559_1681116738103_1_140x140.JPG/dims/resize/Q_80,0'

In [34]:
img = 'https:' + tr.select_one('.cover > img')['src']
img

'https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/083/665/559/83665559_1681116738103_1_140x140.JPG/dims/resize/Q_80,0'

In [36]:
# 19금 노래 제목
tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()

'I AM'

In [37]:
# title
tr.select_one('.title.ellipsis').get_text().strip()

'I AM'

In [38]:
# 위 두 코드를 합치면
# 1)
title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
# 2)
try:
    title = tr.select_one('.title.ellipsis').get_text().strip()
except:
    title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()

In [39]:
# artist
# tr.select_one('.artist.ellipsis').string.strip()
tr.select_one('.artist.ellipsis').get_text().strip()

'IVE (아이브)'

In [40]:
artist = tr.select_one('.artist.ellipsis').string.strip()
artist

'IVE (아이브)'

In [41]:
# album
album = tr.select_one('.albumtitle.ellipsis').text.strip()
album

"I've IVE"

- 한페이내에 있는 정보

In [43]:
# 방법1. 딕셔너리의 리스트로 가져오기
line = []
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0].strip()
    img = 'https:' + tr.select_one('.cover > img')['src']
    try:
        title = tr.select_one('.title.ellipsis').get_text().strip()
    except:
        title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
    artist = tr.select_one('.artist.ellipsis').string.strip()
    album = tr.select_one('.albumtitle.ellipsis').text.strip()
    line.append({'rank':rank,'img':img,'title':title,'artist':artist,'album':album})
df = pd.DataFrame(line)
df.head(3)

,rank,img,title,artist,album
0,1,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,I AM,IVE (아이브),I've IVE
1,2,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,Kitsch,IVE (아이브),I've IVE
2,3,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,꽃,지수 (JISOO),ME


In [44]:
# 방법2. 개별 리스트로 가져오기
rank_list, img_list, title_list, artist_list, album_list = [],[],[],[],[]
for tr in trs:
    rank_list.append(tr.select_one('.number').get_text().split('\n')[0].strip())
    img_list.append('https:' + tr.select_one('.cover > img')['src'])
    try:
        title_list.append(tr.select_one('.title.ellipsis').get_text().strip())
    except:
        title_list.append(tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip())
    artist_list.append(tr.select_one('.artist.ellipsis').string.strip())
    album_list.append(tr.select_one('.albumtitle.ellipsis').text.strip())

In [45]:
df2 = pd.DataFrame({
        'rank': rank_list,
        'img' : img_list,
        'title' : title_list,
        'artist' : artist_list,
        'album' : album_list
})
df2.head(3)

,rank,img,title,artist,album
0,1,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,I AM,IVE (아이브),I've IVE
1,2,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,Kitsch,IVE (아이브),I've IVE
2,3,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,꽃,지수 (JISOO),ME


- 에러 처리

- 모든 페이지에 대해서 처리

In [46]:
# https://www.genie.co.kr/chart/top200?ditc=D&ymd=20230424&hh=14&rtm=Y&pg=2
from datetime import datetime
now = datetime.now()
now

datetime.datetime(2023, 4, 24, 15, 4, 11, 869404)

In [48]:
ymd = now.strftime('%Y%m%d') 
hh = now.strftime('%H') # hh = f'{now.hour:02d}
ymd, hh

('20230424', '15')

In [51]:
from tqdm import tqdm
lines = []
base_url = 'https://www.genie.co.kr/chart/top200?ditc=D&rtm=Y'
for page in tqdm(range(1,5)):
    url = f'{base_url}&ymd={ymd}&hh={hh}&pg={page}'
    result = requests.get(url, headers=header)
    soup = BeautifulSoup(result.text,'html.parser')
    trs = soup.select('#body-content > div.newest-list > div > table > tbody > tr')
    for tr in trs:
        rank = tr.select_one('.number').get_text().split('\n')[0].strip()
        img = 'https:' + tr.select_one('.cover > img')['src']
        try:
            title = tr.select_one('.title.ellipsis').get_text().strip()
        except:
            title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
        artist = tr.select_one('.artist.ellipsis').string.strip()
        album = tr.select_one('.albumtitle.ellipsis').text.strip()
        lines.append({'rank':rank,'img':img,'title':title,'artist':artist,'album':album})
        

100%|██████████| 4/4 [00:02<00:00,  1.38it/s]


In [52]:
df3 = pd.DataFrame(lines)
df3.tail()

,rank,img,title,artist,album
195,196,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,Dive,NCT 도재정,Perfume - The 1st Mini Album
196,197,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,너를 생각해,주시크 (Joosiq),너를 생각해
197,198,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,언제나 사랑해,케이시 (Kassy),조영수 리메이크 프로젝트 Part.2
198,199,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,시작,가호 (Gaho),이태원 클라쓰 OST Part.2 (JTBC 금토드라마)
199,200,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,서랍,10CM,그 해 우리는 OST Part.1


In [60]:
df3.to_csv(f'data/{ymd}-{hh}지니차트.csv',index=False)
pd.read_csv(f'data/{ymd}-{hh}지니차트.csv').head()

,rank,img,title,artist,album
0,1,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,I AM,IVE (아이브),I've IVE
1,2,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,Kitsch,IVE (아이브),I've IVE
2,3,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,꽃,지수 (JISOO),ME
3,4,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,Ditto,NewJeans,NewJeans 'OMG'
4,5,https://image.genie.co.kr/Y/IMAGE/IMG_ALBUM/08...,CHRISTIAN,Zior Park,WHERE DOES SASQUATCH LIVE? PART 1
